### >>>INGESTA DE DATOS

In [79]:
import psycopg2
from psycopg2 import sql
import pandas as pd


# Credenciales de la base de datos
dbname = "DBInmerso"
user = "postgres"
password = "postgres"
host = "localhost"  # Cambia esto según tu configuración
port = "5432"

# Crear la cadena de conexión
connection_string = f"dbname={dbname} user={user} password={password} host={host} port={port}"


try:
    with psycopg2.connect(connection_string) as connection:
        # Consulta para concatenar las tablas
    
        query_concatenar_3 = """SELECT 
                                s.id_solicitud,
                                sc.index_,
                                sc.score, 
                                s.primera_opcion,
                                s.fecha_1op,
                                s.segunda_opcion, 
                                s.fecha_2op,
                                s.tercera_opcion,
                                s.fecha_3op
                                FROM solicitudes s
                                JOIN scoring sc ON s.id_solicitud = sc.id_solicitud;
                                """
        query_concatenar_4 = """SELECT 
                                d.id_hotel,
                                d.fecha_disponibilidad_hab,
                                d.num_hab_disp
                                FROM disponibilidad d;
                                """
        query_concatenar_5 = """SELECT 
                                d.id_hotel,
                                d.fecha_disponibilidad_hab,
                                d.num_hab_disp,
                                d.id_plaza,
                                ho.localizacion
                                FROM disponibilidad d
                                JOIN hoteles ho ON d.id_hotel = ho.id_hotel
                                ;
                                """


        # Crear un DataFrame con los resultados de la consulta concatenada
    
        solicitudes = pd.read_sql_query(query_concatenar_3, connection)
        df_4 = pd.read_sql_query(query_concatenar_4, connection)
        disponibilidad = pd.read_sql_query(query_concatenar_5, connection)

except psycopg2.Error as e:
    print(f"Error al conectar a la base de datos: {e}")


C:\Users\DELL\AppData\Local\Temp\ipykernel_2424\163076696.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  solicitudes = pd.read_sql_query(query_concatenar_3, connection)
C:\Users\DELL\AppData\Local\Temp\ipykernel_2424\163076696.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4 = pd.read_sql_query(query_concatenar_4, connection)
C:\Users\DELL\AppData\Local\Temp\ipykernel_2424\163076696.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  disponibilidad = pd.read_sql_query(query_concatenar_5, connection)


In [81]:
solicitudes 

,id_solicitud,index_,score,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op
0,1170,1,77.500000,Santa Cruz de Tenerife,2024-01-16,Ávila,2024-07-18,Navarra,2024-10-10
1,273,2,74.500000,Barcelona,2024-10-12,Cuenca,2024-05-12,Ávila,2024-12-09
2,991,3,73.142857,Palencia,2024-10-15,Álava,2024-02-10,Sevilla,2024-03-25
3,34207,4,71.000000,Toledo,2024-09-29,Alicante,2024-04-22,Santa Cruz de Tenerife,2024-02-01
4,581,5,70.935897,Lleida,2024-04-18,Guadalajara,2024-06-21,Huesca,2024-11-17
...,...,...,...,...,...,...,...,...,...
1199,54694,1200,11.500000,Soria,2024-08-18,Ávila,2024-06-27,La Coruña,2024-08-14
1200,42361,1201,10.642857,Salamanca,2024-12-11,Salamanca,2024-07-26,Melilla,2024-03-25
1201,32454,1202,9.943223,La Coruña,2024-04-20,Guipúzcoa,2024-01-18,Ávila,2024-01-12
1202,623,1203,7.785714,León,2024-08-16,Burgos,2024-01-27,Ourense,2024-08-21


### >>> FUNCION DE ASIGNACION

In [83]:
# Función para encontrar una coincidencia entre las preferencias y la disponibilidad
def encontrar_coincidencia(preferencia, fecha_preferencia, disponibilidad, plazas_asignadas):
    filtro = (disponibilidad['fecha_disponibilidad_hab'] == fecha_preferencia) & \
             (disponibilidad['localizacion'] == preferencia)
    coincidencias = disponibilidad[filtro]
    
    # Filtrar solo las plazas que aún no han sido asignadas
    coincidencias = coincidencias[~coincidencias['id_plaza'].isin(plazas_asignadas)]
    
    if not coincidencias.empty:
        id_plaza_asignada = coincidencias.iloc[0]['id_plaza']
        
        # Marcar la plaza como asignada en el DataFrame de disponibilidad
        disponibilidad.loc[disponibilidad['id_plaza'] == id_plaza_asignada, 'asignada'] = True
        
        return id_plaza_asignada
    else:
        return None

# Agregar columna 'asignada' en disponibilidad e inicializarla como False
disponibilidad['asignada'] = False

# Aplicar la función para asignar id_plaza a cada id_solicitud respetando el orden de solicitudes
solicitudes['id_plaza_asignada'] = None  # Inicializar la columna
plazas_asignadas = set()  # Conjunto para rastrear plazas ya asignadas

for idx, row in solicitudes.iterrows():
    id_solicitud = row['id_solicitud']
    id_plaza_asignada = encontrar_coincidencia(row['primera_opcion'], row['fecha_1op'], disponibilidad, plazas_asignadas) or \
                        encontrar_coincidencia(row['segunda_opcion'], row['fecha_2op'], disponibilidad, plazas_asignadas) or \
                        encontrar_coincidencia(row['tercera_opcion'], row['fecha_3op'], disponibilidad, plazas_asignadas)

    # Verificar si la plaza ya está asignada o supera la cantidad máxima
    if id_plaza_asignada is not None:
        solicitudes.at[idx, 'id_plaza_asignada'] = id_plaza_asignada
        plazas_asignadas.add(id_plaza_asignada)


solicitudes




,id_solicitud,index_,score,primera_opcion,fecha_1op,segunda_opcion,fecha_2op,tercera_opcion,fecha_3op,id_plaza_asignada
0,1170,1,77.500000,Santa Cruz de Tenerife,2024-01-16,Ávila,2024-07-18,Navarra,2024-10-10,593248
1,273,2,74.500000,Barcelona,2024-10-12,Cuenca,2024-05-12,Ávila,2024-12-09,168904
2,991,3,73.142857,Palencia,2024-10-15,Álava,2024-02-10,Sevilla,2024-03-25,414227
3,34207,4,71.000000,Toledo,2024-09-29,Alicante,2024-04-22,Santa Cruz de Tenerife,2024-02-01,240862
4,581,5,70.935897,Lleida,2024-04-18,Guadalajara,2024-06-21,Huesca,2024-11-17,534208
...,...,...,...,...,...,...,...,...,...,...
1199,54694,1200,11.500000,Soria,2024-08-18,Ávila,2024-06-27,La Coruña,2024-08-14,766204
1200,42361,1201,10.642857,Salamanca,2024-12-11,Salamanca,2024-07-26,Melilla,2024-03-25,919206
1201,32454,1202,9.943223,La Coruña,2024-04-20,Guipúzcoa,2024-01-18,Ávila,2024-01-12,999115
1202,623,1203,7.785714,León,2024-08-16,Burgos,2024-01-27,Ourense,2024-08-21,256711


In [84]:

plazas_asignadas = disponibilidad['asignada'].sum()
plazas_no_asignadas = disponibilidad.shape[0] - plazas_asignadas

print(f'Plazas asignadas: {plazas_asignadas}')
print(f'Plazas no asignadas: {plazas_no_asignadas}')



Plazas asignadas: 1204
Plazas no asignadas: 796


In [85]:
# Verificar si hay id_plaza_asignada repetidos en el DataFrame de solicitudes
repetidos = solicitudes.duplicated(subset='id_plaza_asignada', keep=False)
id_plazas_repetidas = solicitudes.loc[repetidos, 'id_plaza_asignada']

if not id_plazas_repetidas.empty:
    print(f'Hay id_plaza_asignada repetidos!')
    print(id_plazas_repetidas)
else:
    print('No hay id_plaza_asignada repetidos')

No hay id_plaza_asignada repetidos.


In [86]:
disponibilidad.sort_values(by='id_hotel')

,id_hotel,fecha_disponibilidad_hab,num_hab_disp,id_plaza,localizacion,asignada
1352,103223,2024-10-05,1,110394,Lleida,False
961,103223,2024-02-13,1,289721,Lleida,True
855,103223,2024-03-21,1,638228,Lleida,True
1459,103223,2024-02-16,1,950328,Lleida,False
1118,103223,2024-07-18,1,610316,Lleida,True
...,...,...,...,...,...,...
1803,996336,2024-04-23,1,667298,Palencia,False
1971,996336,2024-11-23,1,115251,Palencia,False
328,996336,2024-06-05,1,542759,Palencia,True
590,996336,2024-09-24,1,963300,Palencia,True


In [26]:
for index, row in df_3.iterrows():
    hotel_match = (df_4['id_hotel'] == row['primera_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_1op']) & (df_4['num_hab_disp'] > 0)
    
    if hotel_match.all():
        df_3.loc[index, 'asignacion_hotel'] = row['primera_opcion']
        df_3.loc[index, 'asignacion_fecha'] = row['fecha_1op']
        df_4.loc[hotel_match, 'num_hab_disp'] -= 1
        df_3.loc[index, 'id_resolucion'] = 3
    else:
        hotel_match = (df_4['id_hotel'] == row['segunda_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_2op']) & (df_4['num_hab_disp'] > 0)
        if hotel_match.all():
            df_3.loc[index, 'asignacion_hotel'] = row['segunda_opcion']
            df_3.loc[index, 'asignacion_fecha'] = row['fecha_2op']
            df_4.loc[hotel_match, 'num_hab_disp'] -= 1
            df_3.loc[index, 'id_resolucion'] = 3
        else:
            hotel_match = (df_4['id_hotel'] == row['tercera_opcion']) & (df_4['fecha_disponibilidad_hab'] == row['fecha_3op']) & (df_4['num_hab_disp'] > 0)
            if hotel_match.all():
                df_3.loc[index, 'asignacion_hotel'] = row['tercera_opcion']
                df_3.loc[index, 'asignacion_fecha'] = row['fecha_3op']
                df_4.loc[hotel_match, 'num_hab_disp'] -= 1
                df_3.loc[index, 'id_resolucion'] = 3
            else:
                df_3.loc[index, 'id_resolucion'] = 2